In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Linear

In [4]:
items_df = pd.read_csv("data/items.csv")
reviews_df = pd.read_csv("data/reviews.csv")
users_df = pd.read_csv("data/users.csv")

In [5]:
# Уникальные пользователи
unique_users = users_df['profile_url'].unique()
user2idx = {u: i for i, u in enumerate(unique_users)}

# Уникальные места
unique_items = items_df['detail_id'].unique()
item2idx = {it: i for i, it in enumerate(unique_items)}

num_users = len(unique_users)
num_items = len(unique_items)

print("Число пользователей:", num_users)
print("Число мест:", num_items)

Число пользователей: 3527
Число мест: 17184


In [6]:
edges_user = []
edges_item = []
edge_ratings = []

for _, row in reviews_df.iterrows():
    user_url = row['profile_url']
    item_id = row['detail_id']
    mark = row['mark']

    if user_url in user2idx and item_id in item2idx:
        u_idx = user2idx[user_url]
        i_idx = num_users + item2idx[item_id]

        edges_user.append(u_idx)
        edges_item.append(i_idx)
        edge_ratings.append(mark)

In [7]:
# Преобразуем в тензоры
edge_index = torch.tensor(
    [edges_user + edges_item,
     edges_item + edges_user],
    dtype=torch.long
)

# Нам может понадобиться дублировать рёбра в обе стороны (user->item и item->user),
# если мы хотим работать с неориентированным графом (обычно для GCN так и делаем).
# Для этого мы складываем списки (u->i, i->u).

edge_user_to_item = torch.tensor(
    [edges_user, edges_item],
    dtype=torch.long
)
edge_item_to_user = torch.tensor(
    [edges_item, edges_user],
    dtype=torch.long
)

# Объединим, чтобы получился неориентированный граф
edge_index = torch.cat([edge_user_to_item, edge_item_to_user], dim=1)

edge_ratings_tensor = torch.tensor(edge_ratings + edge_ratings, dtype=torch.float)


In [9]:
users_unified_df = pd.DataFrame({
    "USER_REVIEWER": users_df["REVIEWER"],
    "USER_PHOTO_UPLOADER": users_df["PHOTO_UPLOADER"],
    "ITEM_RATING": 0,
    "ITEM_REVIEWS_COUNT": 0,
    "ITEM_TAG_PEDESTRIAN": 0,
    "ITEM_TAG_MULTI_DAY": 0
})

items_unified_df = pd.DataFrame({
    "USER_REVIEWER": 0,
    "USER_PHOTO_UPLOADER": 0,
    "ITEM_RATING": items_df["rating"],
    "ITEM_REVIEWS_COUNT": items_df["reviews_count"],
    "ITEM_TAG_PEDESTRIAN": items_df["tags_Пешеходные экскурсии"],
    "ITEM_TAG_MULTI_DAY": items_df["tags_Многодневные экскурсии"]
})


In [10]:
# Добавляем колонку 'idx' для сортировки
users_df["idx"] = users_df["profile_url"].map(user2idx)
items_df["idx"] = items_df["detail_id"].map(item2idx)

# user-таблица: сортировка
users_df_sorted = users_df.sort_values("idx")
users_unified_df = users_unified_df.reindex(users_df_sorted.index)
users_unified_df = users_unified_df.reset_index(drop=True)

# item-таблица: сортировка
items_df_sorted = items_df.sort_values("idx")
items_unified_df = items_unified_df.reindex(items_df_sorted.index)
items_unified_df = items_unified_df.reset_index(drop=True)

# Теперь конвертируем в numpy->torch
x_users = torch.tensor(users_unified_df.values, dtype=torch.float)
x_items = torch.tensor(items_unified_df.values, dtype=torch.float)

print("x_users.shape =", x_users.shape)  # (num_users, 6)
print("x_items.shape =", x_items.shape)  # (num_items, 6)


x_users.shape = torch.Size([3527, 6])
x_items.shape = torch.Size([17184, 6])


In [11]:
edges_user = []
edges_item = []
edge_ratings = []

for _, row in reviews_df.iterrows():
    user_url = row["profile_url"]
    item_id = row["detail_id"]
    mark = row["mark"]

    if user_url in user2idx and item_id in item2idx:
        u_idx = user2idx[user_url]
        i_idx = num_users + item2idx[item_id]

        edges_user.append(u_idx)
        edges_item.append(i_idx)
        edge_ratings.append(mark)


In [12]:
user_tensor = torch.tensor(edges_user, dtype=torch.long)
item_tensor = torch.tensor(edges_item, dtype=torch.long)
ratings_tensor = torch.tensor(edge_ratings, dtype=torch.float)

# user->item
edge_user_to_item = torch.stack([user_tensor, item_tensor], dim=0)
# item->user
edge_item_to_user = torch.stack([item_tensor, user_tensor], dim=0)

# Объединим
edge_index = torch.cat([edge_user_to_item, edge_item_to_user], dim=1)
edge_attr = torch.cat([ratings_tensor, ratings_tensor], dim=0)  # дублируем


In [13]:
print("edge_index.shape =", edge_index.shape)  # (2, E*2)
print("edge_attr.shape  =", edge_attr.shape)   # (E*2,)

edge_index.shape = torch.Size([2, 125516])
edge_attr.shape  = torch.Size([125516])


In [14]:
x = torch.cat([x_users, x_items], dim=0)
data = Data(
    x=x,  # (num_users + num_items, num_features)
    edge_index=edge_index,  # (2, 2*E) - неориентированный граф
    edge_attr=edge_attr     # (2*E,) - оценки
)

In [15]:
class GNNRecommender(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, out_dim):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.predict_layer = Linear(out_dim * 2, 1)  # на вход concat(user_emb, item_emb)

    def forward(self, x, edge_index):
        # Первый GCN
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Второй GCN
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        return x  # вернём эмбеддинги всех вершин

    def predict(self, user_emb, item_emb):
        concat = torch.cat([user_emb, item_emb], dim=-1)  # (batch_size, 2*out_dim)
        return self.predict_layer(concat).view(-1)       # (batch_size,)


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GNNRecommender(num_features=x.shape[1], hidden_dim=64, out_dim=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

data = data.to(device)

In [17]:
model.train()
for epoch in range(20):
    optimizer.zero_grad()

    # Получаем эмбеддинги всех вершин
    embeddings = model(data.x, data.edge_index)

    # edge_index[0] — это список вершин-источников, edge_index[1] — вершин-получателей
    u_nodes = data.edge_index[0]
    v_nodes = data.edge_index[1]

    # Вытаскиваем эмбеддинги
    u_emb = embeddings[u_nodes]
    v_emb = embeddings[v_nodes]

    # Предсказываем рейтинг
    pred_ratings = model.predict(u_emb, v_emb)

    # Истинные рейтинги
    true_ratings = data.edge_attr

    loss = loss_fn(pred_ratings, true_ratings)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}, Loss = {loss.item():.4f}")


Epoch 5, Loss = 223.0350
Epoch 10, Loss = 45.9553
Epoch 15, Loss = 18.1348
Epoch 20, Loss = 18.8392


In [18]:
model.eval()
with torch.no_grad():
    all_embeddings = model(data.x, data.edge_index)

In [19]:
test_user_url = "/Profile/yalmaree"
test_user_idx = user2idx[test_user_url]  # число от 0 до num_users-1
test_user_emb = all_embeddings[test_user_idx]  # вектор размером out_dim=32


In [20]:
# Места идут с offset = num_users
item_indices = torch.arange(num_users, num_users + num_items, dtype=torch.long, device=device)
item_embs = all_embeddings[item_indices]

# Сформируем батч из user_emb
# (num_items, out_dim)
test_user_emb_batch = test_user_emb.unsqueeze(0).repeat(item_embs.shape[0], 1)
preds = model.predict(test_user_emb_batch, item_embs)


In [24]:
sorted_indices = torch.argsort(preds, descending=True)
top_k = 5
top_k_indices = sorted_indices[:top_k]

In [25]:
recommended_global_indices = item_indices[top_k_indices]

In [26]:
idx2item = {v: k for k, v in item2idx.items()}

recommended_detail_ids = []
for g_idx in recommended_global_indices.cpu().numpy():
    # item-индекс
    item_local_idx = g_idx - num_users
    detail_id = idx2item[item_local_idx]
    recommended_detail_ids.append(detail_id)

print("Топ-5 рекомендаций для пользователя:", test_user_url)
print(recommended_detail_ids)

Топ-5 рекомендаций для пользователя: /Profile/yalmaree
[300623, 2196855, 300367, 547719, 300366]


In [34]:
import pandas as pd
import torch
import json
from torch_geometric.data import Data

reviews_test_df = pd.read_csv("data/reviews_test.csv")

edges_user_test = []
edges_item_test = []
edge_ratings_test = []

for _, row in reviews_test_df.iterrows():
    user_url = row["profile_url"]
    item_id = row["detail_id"]
    mark = row["mark"]  # предполагаем, что это уже float или int

    if user_url in user2idx and item_id in item2idx:
        u_idx = user2idx[user_url]        # индекс пользователя
        i_idx = num_users + item2idx[item_id]  # индекс места (со сдвигом)

        edges_user_test.append(u_idx)
        edges_item_test.append(i_idx)
        edge_ratings_test.append(mark)

# Превращаем списки в тензоры
user_tensor_test = torch.tensor(edges_user_test, dtype=torch.long)
item_tensor_test = torch.tensor(edges_item_test, dtype=torch.long)
ratings_tensor_test = torch.tensor(edge_ratings_test, dtype=torch.float)

# Дублируем рёбра user->item и item->user
edge_user_to_item_test = torch.stack([user_tensor_test, item_tensor_test], dim=0)
edge_item_to_user_test = torch.stack([item_tensor_test, user_tensor_test], dim=0)

edge_index_test = torch.cat([edge_user_to_item_test, edge_item_to_user_test], dim=1)
edge_attr_test = torch.cat([ratings_tensor_test, ratings_tensor_test], dim=0)

data_test = Data(
    x=x,  # тот же тензор признаков (пользователи + места), что был при обучении
    edge_index=edge_index_test,
    edge_attr=edge_attr_test
).to(device)

model.eval()
with torch.no_grad():
    embeddings_test = model(data_test.x, data_test.edge_index)

# Подготавливаем словарь для получения detail_id по индексу
idx2item = {v: k for k, v in item2idx.items()}

recommendations_dict = {}  # ключ = profile_url, значение = список detail_id

with open("data/test_profile_urls.txt", "r", encoding="utf-8") as f:
    for line in f:
        test_user_url = line.strip()

        if test_user_url in user2idx:
            test_user_idx = user2idx[test_user_url]

            # Эмбеддинг пользователя
            test_user_emb = embeddings_test[test_user_idx]

            # Индексы всех мест [num_users .. num_users + num_items - 1]
            item_indices = torch.arange(
                num_users, num_users + num_items,
                dtype=torch.long, device=device
            )
            # Эмбеддинги для всех мест
            item_embs = embeddings_test[item_indices]

            # Предсказываем рейтинги для всех мест
            test_user_emb_batch = test_user_emb.unsqueeze(0).repeat(item_embs.shape[0], 1)
            preds = model.predict(test_user_emb_batch, item_embs)

            # Сортируем по убыванию, берём топ-10
            sorted_indices = torch.argsort(preds, descending=True)
            top_k = 5
            top_k_indices = sorted_indices[:top_k]

            # Переводим индексы в detail_id
            recommended_global_indices = item_indices[top_k_indices]
            recommended_detail_ids = []
            for g_idx in recommended_global_indices.cpu().numpy():
                item_local_idx = g_idx - num_users
                detail_id = idx2item[item_local_idx]
                # Явно приводим к int (чтобы не было np.int64)
                recommended_detail_ids.append(int(detail_id))

            recommendations_dict[test_user_url] = recommended_detail_ids

        else:
            # Если profile_url нет в словаре
            recommendations_dict[test_user_url] = []

with open("recommendations_gnn.json", "w", encoding="utf-8") as out_file:
    json.dump(recommendations_dict, out_file, ensure_ascii=False, indent=2)

print("Готово! Результат записан в файл recommendations_gnn.json.")

Готово! Результат записан в файл recommendations_gnn.json.
